In [ ]:
from qm import QuantumMachinesManager, QopCaps
from qm.qua import *
from configuration import *

qmm = QuantumMachinesManager(host=qop_ip,port=None, cluster_name=cluster_name)
print(f"send program with config: {qmm.capabilities.supports(QopCaps.config_v2)}")

2025-09-05 21:12:02,713 - qm - INFO     - Starting session: 9d76e44b-d85b-445d-9013-45416bdb3027
2025-09-05 21:12:06,804 - qm - INFO     - Performing health check
2025-09-05 21:12:07,224 - qm - INFO     - Cluster healthcheck completed successfully.
send program with config: True


In [2]:
with program() as hello_qua:
    set_dc_offset("q1_z", "single", 0.153)
    play("saturation", "q1_xy")

In [3]:
# backward compatiblity, you can also use controller config to open qmm
qm = qmm.open_qm(full_config,close_other_machines=True)
job1 = qm.execute(hello_qua) 
job1.wait_until("Done", timeout=60)

2025-09-05 21:12:07,838 - qm - INFO     - Clearing queue
2025-09-05 21:12:08,656 - qm - INFO     - Adding program to queue.
2025-09-05 21:12:10,191 - qm - INFO     - Job c8d35ff0-94db-4374-b90b-a36d5d9c0178 is done


Execucate program with modified full config only change in the job

In [4]:
print("offset before execute:",
qm.get_config()["controllers"][con]['fems'][lf_fem]['analog_outputs'][1]['offset'],"V")

offset before execute: 0.0 V


In [5]:
# temporarily change offset by full config
full_config["controllers"][con]['fems'][lf_fem]['analog_outputs'][1]['offset']=0.5
# execute program with modified full_config
job2 = qm.execute(hello_qua, config=full_config) 
job2.wait_until("Done", timeout=60)

2025-09-05 21:12:11,870 - qm - INFO     - Clearing queue
2025-09-05 21:12:12,764 - qm - INFO     - Adding program to queue.
2025-09-05 21:12:14,299 - qm - INFO     - Job 3deb9b3d-552a-4a3f-9198-625fbe213da5 is done


In [6]:
print("offset after execute:",
qm.get_config()["controllers"][con]['fems'][lf_fem]['analog_outputs'][1]['offset'],"V")

offset after execute: 0.0 V


temporarily change logical config

In [7]:
# change pulse length by logical_config
logical_config['pulses']['saturation_pulse']['length']= 2*saturation_len
# execute program with modified logical_config
job3 = qm.execute(hello_qua, config=logical_config) 
job3.wait_until("Done", timeout=60)

2025-09-05 21:12:15,252 - qm - INFO     - Clearing queue
2025-09-05 21:12:16,130 - qm - INFO     - Adding program to queue.
2025-09-05 21:12:17,674 - qm - INFO     - Job f02920b9-5113-4c3f-b2c3-fb0e4ba5e0de is done


use update_config to change config of qm_open

In [8]:
print("full power before execute:",
controller_config["controllers"][con]['fems'][mw_fem]['analog_outputs'][1]['full_scale_power_dbm'],"dBm")

full power before execute: 1 dBm


In [9]:
# change full scale power dBm for future job by controller_config
controller_config["controllers"][con]['fems'][mw_fem]['analog_outputs'][1]['full_scale_power_dbm']= 4
qm.update_config(controller_config)
# execute program with modified controller_config
job4 = qm.execute(hello_qua, config=controller_config) 
job4.wait_until("Done", timeout=60)

2025-09-05 21:12:18,589 - qm - INFO     - Clearing queue
2025-09-05 21:12:19,716 - qm - INFO     - Adding program to queue.
2025-09-05 21:12:21,251 - qm - INFO     - Job 6f1e190f-8cb0-4948-afe0-6c875f7504c6 is done


In [10]:
print("full power after execute:",
controller_config["controllers"][con]['fems'][mw_fem]['analog_outputs'][1]['full_scale_power_dbm'],"dBm")

full power after execute: 4 dBm


In [11]:
qm.close()

True